<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/Fund2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
%%capture
!pip install node2vec
!pip install pyvis

In [2]:
import pandas as pd 
import numpy as np


import networkx as nx
from node2vec import Node2Vec
import gensim
from gensim.models.word2vec import Word2Vec

## 整0.1資料

In [3]:
fund_code_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/FUND_CODE_DATA.csv', encoding='CP950')
#fund_code_df = pd.read_csv('FUND_CODE_DATA.csv', encoding='CP950')

fund_code_df = fund_code_df[['產品代碼', '基金名稱', 'ISIN Code']]

In [21]:
fund_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/fund_desc.csv', encoding='CP950')
#fund_df = pd.read_csv('fund_desc.csv', encoding='CP950')

## 只取基金名稱主體
fund_df['CNAME_short'] = fund_df['CNAME'].str.replace('\（[^（）]*\）', '', regex=True)


In [22]:
fund_df[['CNAME', 'CNAME_short']].head(3)

,CNAME,CNAME_short
0,聯博新興市場債券基金ＥＡ（美元）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關費用）（...,聯博新興市場債券基金ＥＡ
1,聯博新興市場債券基金ＥＡ（澳幣避險）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關費用...,聯博新興市場債券基金ＥＡ
2,聯博－新興市場債券基金ＥＡ（南非幣避險）（穩定月配息）（本基金進行配息前未先扣除應負擔之相關...,聯博－新興市場債券基金ＥＡ


In [23]:
index = ['FUND_CD']

cate_list = [
 'RISK_LEVEL_ALL',
 'FUNDCRYID',
 'RISKLEVEL',
 '配息',
 '股債分類',
 '基金類型',
 '產品類型',
 '基金分類',
 'SITCA分類',
 #'SITCA細類',
 #'追蹤指標',
 '境外內',
 '前後收'            
]


weight_list = [
 '股票比',
 '債券比',
 '現金比',
 '其他比',
 '成熟股',
 '成熟股_美國',
 '成熟股_北美',
 '成熟股_歐洲',
 '成熟股_英',
 '成熟股_日本',
 '成熟股_亞太',
 '新興股',
 '新興股_拉美',
 '新興股_新歐',
 '新興股_非洲',
 '新興股_中東',
 '新興股_新亞',
 '原物料',
 '非必消',
 '金融',
 '房地產',
 '電信',
 '能源',
 '工業',
 '科技',
 '必需消',
 '健護',
 '公用事業',
 '成熟債',
 '成熟債_美國',
 '成熟債_北美',
 '成熟債_歐洲',
 '成熟債_英',
 '成熟債_日本',
 '成熟債_亞太',
 '新興債',
 '新興債_拉美',
 '新興債_新歐',
 '新興債_非洲',
 '新興債_中東',
 '新興債_新亞',
 'AAA',
 'AA',
 'A',
 'BBB',
 'BB',
 'B',
 'B以下',
 '無信評'
]

all_list = index+cate_list+weight_list

In [25]:
# 保留原始資料
fund_org_df = fund_df.copy() 
fund_df[cate_list] = fund_df[cate_list].astype(str)

# 取有風險等級
fund_df = fund_df[fund_df['風屬']!='nan']

# 取出需要欄位
fund_df = fund_df[all_list]
fund_df = fund_df.replace('nan', np.nan)

In [26]:
fund_org_df.shape

(2559, 250)

In [27]:
fund_df[weight_list] = fund_df[weight_list].astype(float).fillna(0)

# 權重原是百分比
fund_df[weight_list] = fund_df[weight_list]/100

## Graph創立


In [39]:
## 製作graph

## 將類別型變數轉為one hot encoding 以利轉為graph
fund_edge = pd.get_dummies(
  fund_df[all_list], 
  columns=cate_list, prefix='', prefix_sep=''
)

In [40]:
fund_edge.head(2)

,FUND_CD,股票比,債券比,現金比,其他比,成熟股,成熟股_美國,成熟股_北美,成熟股_歐洲,成熟股_英,成熟股_日本,成熟股_亞太,新興股,新興股_拉美,新興股_新歐,新興股_非洲,新興股_中東,新興股_新亞,原物料,非必消,金融,房地產,電信,能源,工業,科技,必需消,健護,公用事業,成熟債,成熟債_美國,成熟債_北美,成熟債_歐洲,成熟債_英,成熟債_日本,成熟債_亞太,新興債,新興債_拉美,新興債_新歐,新興債_非洲,...,金融,韓國股市,香港股市,高收益債,債券型,平衡型,股票型,貨幣型,亞洲太平洋債券,亞洲太平洋股票,北美債券,北美股票,台灣小型公司股票,台灣平衡,台灣股票,台灣資訊科技股票,商品及能源股票,單一國家股票,大中華股票,小型公司股票,新興市場股票,日本股票,歐洲債券,歐洲股票,環球債券,環球平衡,環球股票,生物科技\健康護理股票,貨幣市場,資訊科技股票,金融股票,其他型,固定收益型,平衡型(混合型),股票型,貨幣市場型,境內,境外,前收,後收
0,AOB4,0.0,0.9891,0.01,0.0009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1198,0.0018,0.053,0.0321,0.01,0.0,0.0229,0.8643,0.3529,0.1078,0.1782,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1
1,AOB5,0.0,0.9891,0.01,0.0009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1198,0.0018,0.053,0.0321,0.01,0.0,0.0229,0.8643,0.3529,0.1078,0.1782,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1


In [41]:
# 以利轉為graph
fund_edge = fund_edge.set_index('FUND_CD').stack()
fund_edge = fund_edge[fund_edge>0]
fund_edge = fund_edge.reset_index().values

In [42]:
# 用這格式一個一個新增結點與權重
fund_edge

array([['AOB4', '債券比', 0.9891],
       ['AOB4', '現金比', 0.01],
       ['AOB4', '其他比', 0.0009],
       ...,
       ['WAB1', '股票型', 1.0],
       ['WAB1', '境外', 1.0],
       ['WAB1', '前收', 1.0]], dtype=object)

In [43]:
G = nx.Graph()

for x in list(fund_edge):
  G.add_edge(x[0], x[1], weight=x[2])

In [ ]:

## 拿來簡單畫圖的graph
plot_G = nx.Graph()

for x in list(fund_edge[0:100]):
  plot_G.add_edge(x[0], x[1], weight=x[2])

In [ ]:
%%capture
# 用pyvis畫圖

from pyvis.network import Network
nt = Network("500px", "1000px", notebook=False)
nt.from_nx(plot_G)


In [ ]:

from IPython.core.display import display, HTML

nt.show('network.html')
display(HTML('network.html'))


# Node2Vec序列生成

In [ ]:
%%time
'''
:param graph: Input graph
:param dimensions: Embedding dimensions (default: 128)
:param walk_length: Number of nodes in each walk (default: 80)
:param num_walks: Number of walks per node (default: 10)
:param p: Return hyper parameter (default: 1)
:param q: Inout parameter (default: 1)
:param weight_key: On weighted graphs, this is the key for the weight attribute (default: 'weight')
:param workers: Number of workers for parallel execution (default: 1)
:param sampling_strategy: Node specific sampling strategies, supports setting node specific 'q', 'p', 'num_walks' and 'walk_length'.
:param seed: Seed for the random number generator.
'''

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**

# 生成序列
node2vec = Node2Vec(G, dimensions=64, walk_length=50, num_walks=20, workers=1, seed=9527)  # Use temp_folder for big graphs

Computing transition probabilities:   0%|          | 0/2044 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 20/20 [02:22<00:00,  7.14s/it]

CPU times: user 5min 15s, sys: 2.4 s, total: 5min 17s
Wall time: 5min 15s


In [ ]:
node2vec.walks[0]

['FDU1',
 '4',
 'AOA7',
 '非必消',
 'MSG9',
 '資訊科技股票',
 'MSH1',
 'STOCK',
 'ALM7',
 '科技',
 'FRP5',
 '股票型',
 'JAA8',
 'STOCK',
 'SCO1',
 '境外',
 'JFL1',
 '境外',
 'JFU8',
 '股票比',
 '3914',
 'STOCK',
 'FDM1',
 '成熟股',
 'JAA4',
 '成熟股',
 '3635',
 '4',
 '8111',
 '4',
 'FDS5',
 'STOCK',
 'FDK7',
 '成熟股_歐洲',
 'SCR2',
 '歐洲股票',
 'AEA4',
 '股票比',
 '6202',
 '後收',
 'FFA6',
 '北美股票',
 'UBA5',
 '工業',
 '1336',
 '新興股',
 'KBA1',
 '成熟股_北美',
 '7799',
 '環球股票']

# Embedding訓練

In [ ]:
node_corpus = node2vec.walks

In [ ]:
%%time
n2v_model = Word2Vec(node_corpus, size=256, iter=5, window=8, sg=1, negative=10, min_count=1, workers=1)

CPU times: user 2min 18s, sys: 193 ms, total: 2min 18s
Wall time: 2min 17s


In [ ]:
n2v_model.save("fund_node2vec.model")

In [ ]:
## 取出市場向量
n2v_mkt_emb = {x:n2v_model.wv[x] for x in fund_df['基金類型'].unique()}

輸出成embedding projector可以載入的格式

https://projector.tensorflow.org/

In [ ]:
## 市場向量

np.savetxt("n2v_mkt_emb.tsv", n2v_model.wv[fund_df['基金類型'].unique()], delimiter="\t")
pd.DataFrame({'mkt':fund_df['基金類型'].unique()}).to_csv('n2v_mkt_emb_meta.tsv', sep = '\t',index=False, header=False)

In [ ]:
## 基金細項向量

np.savetxt("n2v_fund_emb.tsv", n2v_model.wv[fund_df['FUND_CD']], delimiter="\t")
fund_org_df[['CNAME_short','RISK_LEVEL_ALL','FUNDCRYID','產品類型','基金類型','基金分類',]].to_csv('n2v_fund_emb_meta.tsv', sep = '\t',index=False, header=False)

In [ ]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

# Pretrained W2V (待研究)

嘗試提取財金語料pretrained的字詞提取embedding

In [ ]:
# 下載英文財金語料庫pretrained fasttext word2vec
!wget https://www.rahimikia.com/FinText/FinText_FastText_Skip-gram.zip
!unzip /content/FinText_FastText_Skip-gram.zip

In [ ]:
from gensim.models import Word2Vec
from gensim.models import FastText
import gensim.downloader as api 

In [ ]:
FinText_Word2Vec_skipgram = Word2Vec.load('/content/FinText_FastText_Skip-gram/Word_Embedding_2000_2015')

In [ ]:

prod=['fund',
 'etf',
 'stock',
 'preferred_stock',
 'auto_loan',
 'trust',
 'life_insurance',
 'insurance',
 'personal_loan',
 'mortgage',
 'loan',
 'treasury_bond',
 'foreign_exchange',
 'online_banking',
 'fintech',
 'blockchain',
 'bitcoin',
 'robo_advisor'
 ]

In [ ]:
product_embedding = FinText_Word2Vec_skipgram.wv[prod]

In [ ]:
product_embedding

array([[ 0.14604235,  0.1704098 ,  0.01929267, ..., -0.13392453,
        -0.3286863 ,  0.01605617],
       [ 0.40752319,  0.3141233 , -0.22489968, ..., -0.28437993,
         0.02246348,  0.17138132],
       [ 0.09119733,  0.13523722,  0.01482047, ..., -0.09712761,
        -0.23065072, -0.22710954],
       ...,
       [-0.00303637,  0.145203  , -0.41418603, ..., -0.11681274,
        -0.14459747, -0.02398938],
       [-0.07647095,  0.24916057, -0.12600106, ...,  0.3001527 ,
        -0.21294053, -0.04931631],
       [ 0.29539302, -0.00322265,  0.12641694, ...,  0.13217312,
        -0.4011273 , -0.07066225]], dtype=float32)